In [1]:
import pandas as pd
import numpy as np

In [5]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
train_df.set_index('id', inplace=True)
train_df.head()

,Unnamed: 0,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,monthly_revenue
id,,,,,,,,,,,,,,,,,,,,,
19792418,879,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,f,3,...,4.80,4.82,4.90,4.87,4.69,4.81,f,3,0.77,2108
1015650685503221866,6416,Guest suite in Vancouver · ★New · 2 bedrooms ·...,NaN,139792573,Daniel,within a few hours,100%,100%,f,1,...,NaN,NaN,NaN,NaN,NaN,NaN,f,1,NaN,2730
35265562,1820,Guest suite in Vancouver · ★4.85 · 2 bedrooms ...,Beautiful neighbourhood close to prosperous Ma...,265504225,Alex,within an hour,100%,98%,t,1,...,4.90,4.78,4.97,4.94,4.90,4.75,f,1,3.22,2254
911948980885194155,5346,Home in Vancouver · ★5.0 · 1 bedroom · 1 bed ·...,We are located in a quiet residential neighbor...,22595056,Raymond,NaN,NaN,92%,t,1,...,5.00,5.00,5.00,5.00,4.86,5.00,f,1,1.28,3187
46069251,2484,Guest suite in Vancouver · ★4.93 · 1 bedroom ·...,Kitsilano at it's best! Short walk to all the ...,65683877,Yendi,within an hour,100%,95%,t,2,...,4.93,4.89,4.97,4.97,4.96,4.85,f,1,2.01,3479


In [13]:
train_df['name'].iloc[0].split('.')

['Home in Vancouver · ★4', '75 · 1 bedroom · 1 bed · 1 bath']